### Assignment 2: Text Classification

In [1]:
%load_ext autoreload
%autoreload 2
import os
import csv
from random import seed
from pathlib import Path
from itertools import chain
import torch
from tqdm import tqdm
from IPython.display import HTML, display
import gensim.downloader as api
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pack_sequence
from data_preprocessing import read_data,read_test, Tokenizer, TextDataset,\
    Vocab,train_test_split
from utils import show_example
from model import prepare_emb_matrix, RecurrentClassifier
from trainer import Trainer

Toxicity, such as insults, threats and hate speech, in online conversations is a real threat to productive sharing of opinons. To mitigate this problem automatic comment filtering system may be applied.
In this assignment you are provided with data, collected by [Jigsaw](https://jigsaw.google.com/
) company from Wikipedia’s talk page edits. Each comment was labeled with toxicity rating from 0 to 5. Here are some examples of the least toxic comments. 

In [2]:
data = read_data()
seed(4)

# change this at your own risk 
ratings_to_show = (0, 1, 2)
display(HTML(show_example(data, ratings=ratings_to_show)))       

Toxicity,Comment,😇,this was an excellent application of wp summary i was out getting bad pictures of the dangling crane over th street so i didn t do the split but it is very welcome and should be followed by neatening the division after election day when the usa has other things to think about we can more carefully trim silliness but i don t think we ll trim so much that making it all one big article will be...,😐,sorry i just went through and took out a lot of junk that people put that he was a penis toucher that he was fat and that his wife s name was cristal and i noticed that someone took out his bi sex quote you that got me mad because it s just so silly to argue over what the man said i meant you no ill will,😧,how rude seriously my sillypeppymacspeed page was necessary and i don t fuck ing appreciate you flagging it meet me face to face in detroit and we ll settle this out come on homeboy come fite me i m a real gangster i bet ur trap cat


The task is to build a classifier system. Let's create a baseline recurrent model. We'll start with building a vocabulary

In [3]:
# Press shift-tab to check docstrings
tok = Tokenizer()
tok_texts = [tok.tokenize(t) for t in chain(*data.values())]
vocab = Vocab(tok_texts, max_vocab_size=30000)

Then data is splitted into train and validation parts 

In [4]:
train_texts, train_labels, val_texts, val_labels = train_test_split(data)
train_dataset = TextDataset([tok.tokenize(t) for t in train_texts], train_labels, vocab)
val_dataset = TextDataset([tok.tokenize(t) for t in val_texts], val_labels, vocab)

Then pretrained embeddings are obtained with Gensim - it'll automatically download them for you. [Here](https://github.com/RaRe-Technologies/gensim-data#models
) you can see other pretrained embeddings.

In [5]:
# store embeddings in current directory
os.environ["GENSIM_DATA_DIR"] = str(Path.cwd())
# will download embeddings or load them from disk
gensim_model = api.load("glove-wiki-gigaword-100")
emb_matrix = prepare_emb_matrix(gensim_model, vocab)

Now let's define hyperparameters for our baseline model. It'll be a 2-layered unidiractional LSTM

In [6]:
torch.cuda.is_available()

True

In [ ]:
config = {
    "freeze": True,
    "cell_type": "LSTM",
    "cell_dropout": 0.3,
    "num_layers": 2,
    "hidden_size": 128,
    "out_activation": "relu",
    "bidirectional": False,
    "out_dropout": 0.2,
    "out_sizes": [200],
}

trainer_config = {
    "lr": 3e-4,
    "n_epochs": 10,
    "weight_decay": 1e-6,
    "batch_size": 128,
    "device": "cuda" if torch.cuda.is_available() else "cpu"
}
clf_model = RecurrentClassifier(config, vocab, emb_matrix)

In [7]:
#ТЕСТОВЫЕ КОНФИГ

config = {
    "freeze": True,
    "cell_type": "GRU",
    "cell_dropout": 0.2,
    "num_layers": 3,
    "hidden_size": 512,
    "out_activation": "relu",
    "bidirectional": False,
    "out_dropout": 0.2,
    "out_sizes": [200],
}

trainer_config = {
    "lr": 3e-4,
    "n_epochs": 10,
    "weight_decay": 1e-6,
    "batch_size": 128,
    "device": "cuda" if torch.cuda.is_available() else "cpu"
}
clf_model = RecurrentClassifier(config, vocab, emb_matrix)

In [8]:
train_dataloader = DataLoader(train_dataset, 
                              batch_size=trainer_config["batch_size"],
                              shuffle=True,
                              num_workers=8,
                              collate_fn=train_dataset.collate_fn)
val_dataloader = DataLoader(val_dataset, 
                            batch_size=trainer_config["batch_size"],
                            shuffle=False,
                            num_workers=8,
                            collate_fn=val_dataset.collate_fn)
t = Trainer(trainer_config)
t.fit(clf_model, train_dataloader, val_dataloader)

RuntimeError: CUDA error: unspecified launch failure

Let's save model, load it from checkpoint and check on some commments

In [9]:
t.save("baseline_model.ckpt")

RuntimeError: cuda runtime error (719) : unspecified launch failure at C:\w\1\s\windows\pytorch\torch/csrc/generic/serialization.cpp:23

In [ ]:
t = Trainer.load("baseline_model.ckpt")

In [ ]:
def predict_toxicity(model, comment):
    tok_text = tok.tokenize(comment)
    indexed_text = torch.tensor(vocab.vectorize(tok_text)).to(t.device)
    rating = model(pack_sequence([indexed_text])).argmax().item()
    print(f"Toxicity rating for \"{comment}\" is: {rating}") 

predict_toxicity(t.model, "Please sir do not delete my edits")
predict_toxicity(t.model, "They are nazi pal, forget it")
predict_toxicity(t.model, "You suck")

Now let's prepare a submission file

In [ ]:
test_uuids, test_texts = read_test("data/test.csv")
test_dataloader = DataLoader( TextDataset([tok.tokenize(t) for t in test_texts], [-1] * len(test_texts), vocab), 
                            batch_size=trainer_config["batch_size"],
                            shuffle=False,
                            num_workers=8,
                            collate_fn=val_dataset.collate_fn)

predictions = t.predict(test_dataloader)

In [10]:
#predictions = t.predict(val_dataloader)

RuntimeError: CUDA error: unspecified launch failure

In [ ]:
from sklearn.metrics import accuracy_score
#accuracy_score(val_labels, predictions)

In [ ]:
def save_test_predictions(test_predictions, path):
    assert len(test_predictions) == len(test_texts)
    with open(path, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile, delimiter=',')
        writer.writerow(["uuid","comment_text","toxicity"])
        for uuid, text, pred in zip(test_uuids, test_texts, test_predictions):
            writer.writerow([uuid, text, pred])
        
save_test_predictions(predictions, "./best_results.csv")

# Evaluation

Note that generally not all errors are equal: for example, predicting score 0 while the target is 5 is seemingly worse than predicting 3 instead of 4. That's why your model will be evaluated by [MSE](https://en.wikipedia.org/wiki/Mean_squared_error) loss used for regression tasks. Also note that your model is not required to predict integers, scores like 0.31, 2.718 etc. are fine too. Good luck!

### Optional part: automatic hyperparameter tuning with Optuna

Optuna is a framework which lets you easily tweak hypermarameters of your model. In this part we'll use it to improve quality on validation data - we'll select model with best accuracy.

In [ ]:
from optuna import create_study
from pprint import pprint


BEST_ACC = 0.0

def objective(trial):
    global BEST_ACC
    
    n_hidden_layers = trial.suggest_int("n_hidden_layers", 0, 3)
    hidden_layer_size = trial.suggest_int("hidden_layer_size", 10, 1000)
    
    config = {
        "freeze": True,
        "cell_type": trial.suggest_categorical("cell_type", ["RNN", "LSTM", "GRU"]),
        "cell_dropout": trial.suggest_loguniform("cell_dropout", 1e-9, 0.9),
        "num_layers": trial.suggest_int("num_layers", 1, 3),
        "hidden_size": trial.suggest_int("hidden_size", 10, 1000),
        "out_activation": trial.suggest_categorical("out_activation", 
                                                    ["sigmoid", "tanh", "relu", "elu"]),
        "bidirectional": trial.suggest_categorical("bidirectional", [True, False]),
        "out_dropout": trial.suggest_loguniform("out_dropout", 1e-9, 0.9),
        "out_sizes": [hidden_layer_size] * n_hidden_layers,
    }

    trainer_config = {
        "lr": trial.suggest_loguniform("lr", 1e-5, 1e-3),
        "n_epochs": 10,
        "weight_decay": trial.suggest_loguniform("weight_decay", 1e-9, 1e-1),
        "batch_size": 128,
        "device": "cuda" if torch.cuda.is_available() else "cpu",
        "verbose": False,
    }
    
    pprint({**config, **trainer_config})
        
    clf_model = RecurrentClassifier(config, vocab, emb_matrix)
    t = Trainer(trainer_config)
    t.fit(clf_model, train_dataloader, val_dataloader)
    val_acc =  t.history["val_acc"][-1]
    if val_acc > BEST_ACC:
        BEST_ACC = val_acc
        t.save("optuna_model.ckpt")
    return val_acc

In [ ]:
study = create_study(direction="maximize")
# you can set more trials
study.optimize(objective, n_trials=10)

### Final prediction

In [ ]:
t = Trainer.load("optuna_model.ckpt")
save_test_predictions(t.predict(test_dataloader), "./best_results_hparam_search.csv")

In [ ]:
t = Trainer.load("optuna_model.ckpt")
save_test_predictions(t.predict(test_dataloader), "./best_results_hparam_search.csv")

In [ ]:
study = create_study(direction="maximize")
# you can set more trials
study.optimize(objective, n_trials=10)